## Homework 10 

#### Building the model 

In [1]:
!docker build -t zoomcamp-model:v001 .

#1 [internal] load build definition from Dockerfile
#1 sha256:7615ba0559e63009004b56917cf335503ebe41c24025b0b9f5de2595ac87f381
#1 transferring dockerfile: 32B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:19a7ffb08868ac88b76282a87cfd164a43db5c6615da852b86997d51d4412a97
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/svizor/zoomcamp-model:3.9.12-slim
#3 sha256:6faf31e8764c50c3c410067d53ed2ce2fdd92f64122b1156114fff652d286c85
#3 DONE 0.0s

#4 [1/5] FROM docker.io/svizor/zoomcamp-model:3.9.12-slim
#4 sha256:1f36ffbaefbdf338e5fa88524f56f26d17e244b60e5e6eb69d6334a4c011bbf4
#4 DONE 0.0s

#6 [internal] load build context
#6 sha256:145b92eb4f06eeb04498e013967649a82de5f7720c5e16a2fdb4675b0e982d3c
#6 transferring context: 17.02kB 0.0s done
#6 DONE 0.0s

#8 [4/5] RUN pipenv install --system --deploy
#8 sha256:c3333bc1d32eac4a9348950bde03d434d72975835da8c571929d0783c0e52e31
#8 CACHED

#7 [3/5] COPY [Pipfile, Pipfile.lock, ./]
#7 sha256:1bfa30

### Question 1

In [2]:
!docker run -it --rm -p 9696:9696 zoomcamp-model:v001

the input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'


In [5]:
!python q6_test.py

{'get_card': True, 'get_card_probability': 0.7692649226628628}


### Question 2

In [6]:
!kind --version

kind version 0.17.0


####  Creating a Cluster

In [7]:
!kind create cluster

ERROR: failed to create cluster: node(s) already exist for a cluster with the name "kind"


In [8]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:61902
CoreDNS is running at https://127.0.0.1:61902/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


### Question 4

In [9]:
!kubectl get service

NAME          TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
credit-card   LoadBalancer   10.96.161.222   <pending>     80:31263/TCP   18h
kubernetes    ClusterIP      10.96.0.1       <none>        443/TCP        20h


### Question 5

In [11]:
!kind load docker-image zoomcamp-model:v001

Image: "zoomcamp-model:v001" with ID "sha256:ff934ed6599ef0b07b02a007780e69a2dd2c903740f1b5c8dd02e27093a30375" found to be already present on all nodes.


### Question 6

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: credit-card
    spec:
      selector:
        matchLabels:
          app: credit-card
      replicas: 1
      template:
        metadata:
          labels:
            app: credit-card
        spec:
          containers:
          - name: credit-card
            image: zoomcamp-model:v001
            resources:
              requests:
                memory: "64Mi"
                cpu: "100m"            
              limits:
                memory: "128Mi"
                cpu: "100m"
            ports:
            - containerPort: 9696

### Question 7

    apiVersion: v1
    kind: Service
    metadata:
      name: credit-card
    spec:
      type: LoadBalancer
      selector:
        app: credit-card
      ports:
      - port: 80
        targetPort: 9696

#### Testing the service 

In [15]:
!python q6_test.py

{'get_card': True, 'get_card_probability': 0.7692649226628628}


#### Autoscaling

In [16]:
!kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3

Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "credit-card-hpa" already exists


In [17]:
!kubectl get hpa

NAME              REFERENCE                TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
credit-card-hpa   Deployment/credit-card   <unknown>/20%   1         3         1          18h


In [20]:
import requests
import time


url = "http://localhost:9696/predict"

client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
while True:
    time.sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)

{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226628628}
{'get_card': True, 'get_card_probability': 0.7692649226

KeyboardInterrupt: 

### Question 8

In [19]:
!kubectl get hpa credit-card-hpa --watch

^C
